<a href="https://colab.research.google.com/github/meghaaroraa/hindi-sign-language-recognition/blob/main/sign_language_using_cnn_and_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir = '/content/drive/MyDrive/SIGN LANGUAGE - SUMMER INTERNSHIP/DATASET/train'
test_dir = '/content/drive/MyDrive/SIGN LANGUAGE - SUMMER INTERNSHIP/DATASET/test'

In [ ]:
import os
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Define model parameters
num_classes = 43
input_shape = (64, 64, 3)

In [ ]:
# Data preprocessing and augmentation
train_data = keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=10,  # Augmentation: randomly rotate images by 10 degrees
    width_shift_range=0.1,  # Augmentation: randomly shift images horizontally by 10% of the width
    height_shift_range=0.1,  # Augmentation: randomly shift images vertically by 10% of the height
    shear_range=0.1,  # Augmentation: randomly apply shearing transformations
    zoom_range=0.1,  # Augmentation: randomly zoom into images
)
test_data = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Load and augment training data
train_generator = train_data.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode='categorical'
)

Found 1186 images belonging to 43 classes.


In [ ]:
# Load and rescale test data (without augmentation)
test_generator = test_data.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode='categorical'
)

Found 317 images belonging to 43 classes.


In [ ]:
# Create the CNN model
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Reshape((-1, 64)),  # Reshape the output of the convolutional layers to fit LSTM
    layers.LSTM(64, return_sequences=True),  # Add LSTM layer with 64 units
    layers.LSTM(32),  # Add LSTM layer with 32 units

    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

In [ ]:
# Save the trained model
model.save('sign_language_model.h5')

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Enable eager execution
tf.config.run_functions_eagerly(True)

model = keras.models.load_model('/content/sign_language_model.h5')

In [ ]:
def get_ground_truth_label(filename):
    # Extract the class or category information from the filename
    label = filename.split('_')[0]

    # Map the label to a numeric value if needed
    label_mapping = {'अ': 0, 'आ': 1, 'इ': 2, 'ई':3, 'उ':4, 'ऊ':5, 'ए':6, 'ऐ':7, 'ओ':8, 'औ':9,
           'क':10, 'क्ष':11, 'ख':12, 'ग':13, 'घ':14, 'च':15, 'छ':16, 'ज':17, 'ज्ञ':18, 'झ':19, 'ट':20,
            'ठ':21, 'ड':22, 'ढ':23, 'ण':24, 'त':25, 'थ':26, 'द':27, 'ध':28, 'न':29, 'प':30,
           'फ':31, 'ब':32, 'भ':33, 'म':34, 'य':35, 'र':36, 'ल':37, 'ळ':38, 'व':39, 'श':40, 'स':41, 'ह':42}

    # Return the ground truth label
    return label_mapping[label]

In [ ]:
import os
import numpy as np
from PIL import Image

# Set the path to the parent directory containing the image folders
parent_dir = '/content/drive/MyDrive/SIGN LANGUAGE - SUMMER INTERNSHIP/DATASET/test'

# Initialize variables for overall accuracy calculation
total_images = 0
total_correct_predictions = 0

# Iterate over the folders
for folder_name in os.listdir(parent_dir):
    folder_path = os.path.join(parent_dir, folder_name)

    # Initialize variables for folder-wise accuracy calculation
    folder_images = 0
    folder_correct_predictions = 0

    # Iterate over the images in the current folder
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)

        # Load and preprocess the image
        image = Image.open(image_path)
        image = image.resize((64, 64))  # Adjust the size based on your model's input requirements
        image = np.array(image) / 255.0  # Normalize the pixel values

        # Make a prediction
        prediction = model.predict(np.expand_dims(image, axis=0))
        predicted_label = np.argmax(prediction)

        # Compare with ground truth label
        ground_truth_label = get_ground_truth_label(filename)  # Implement your own function to get the ground truth label
        if predicted_label == ground_truth_label:
            folder_correct_predictions += 1

        folder_images += 1

    folder_accuracy = folder_correct_predictions / folder_images
    print(f"Accuracy for folder {folder_name}: {folder_accuracy}")

    total_images += folder_images
    total_correct_predictions += folder_correct_predictions

overall_accuracy = total_correct_predictions / total_images
print(f"Overall accuracy: {overall_accuracy}")

1/1 [==============================] - 0s 131ms/step
Accuracy for folder क्ष: 1.0
1/1 [==============================] - 0s 151ms/step
Accuracy for folder ळ: 1.0
1/1 [==============================] - 0s 120ms/step
Accuracy for folder ज्ञ: 1.0
1/1 [==============================] - 0s 174ms/step
Accuracy for folder आ: 1.0
1/1 [==============================] - 0s 119ms/step
Accuracy for folder अ: 1.0
1/1 [==============================] - 0s 121ms/step
Accuracy for folder ई: 0.8888888888888888
1/1 [==============================] - 0s 127ms/step
Accuracy for folder इ: 0.4
1/1 [==============================] - 0s 145ms/step
Accuracy for folder ए: 0.6666666666666666
1/1 [==============================] - 0s 127ms/step
Accuracy for folder ऊ: 0.8
1/1 [==============================] - 0s 124ms/step
Accuracy for folder उ: 1.0
1/1 [==============================] - 0s 189ms/step
Accuracy for folder औ: 1.0
1/1 [==============================] - 0s 177ms/step
Accuracy for folder ओ: 0.71428571